In [26]:
from pyspark.sql.functions import udf, struct
from pyspark.sql.types import IntegerType
from sparkstudy.deploy.demo_sessions import DemoSQLSessionFactory
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


舒适化数据

In [21]:
sessionFactory = DemoSQLSessionFactory(name="local file")
spark = sessionFactory.build_session()
data = [("1","a","e"),("2","b","f"),("3","c","g")]
columns = ["str_age","name","address"]
logData = spark.createDataFrame(data, columns).cache()
logData.show()

+-------+----+-------+
|str_age|name|address|
+-------+----+-------+
|      1|   a|      e|
|      2|   b|      f|
|      3|   c|      g|
+-------+----+-------+



下面这个例子。是一个读一列的例子。
[例子来源](https://www.bmc.com/blogs/how-to-write-spark-udf-python/)

In [22]:
colsInt = udf(lambda z: to_int(z), IntegerType())
spark.udf.register("colsInt", colsInt)
def to_int(s):
    if isinstance(s, str):
        return int(s)
    else:
         return None

In [23]:
df2 = logData.withColumn('age',colsInt('str_age'))
df2.show()

+-------+----+-------+---+
|str_age|name|address|age|
+-------+----+-------+---+
|      1|   a|      e|  1|
|      2|   b|      f|  2|
|      3|   c|      g|  3|
+-------+----+-------+---+



读多列的样子

In [30]:
def count_columns(row):
    for r in row:
        print(r)
    return len(row)
countRow = udf(lambda row: count_columns(row), IntegerType())

In [33]:
columns = struct('name','address')
df3 = logData.withColumn("columns", countRow(columns))
df3.show()

+-------+----+-------+-------+
|str_age|name|address|columns|
+-------+----+-------+-------+
|      1|   a|      e|      2|
|      2|   b|      f|      2|
|      3|   c|      g|      2|
+-------+----+-------+-------+

